**Importing all the required libraries**

In [0]:
import csv
import os
from random import seed
from random import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, MaxPooling2D, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import softmax, sigmoid
from tensorflow.python.keras.optimizers import SGD
import tensorflow as tf

**Rotating the images to increase the size of the dataset**

In [0]:
# import PIL
# from PIL import Image
# import cv2
# image_paths = ["/content/drive/My Drive/data/with_mask", "/content/drive/My Drive/data/without_mask"]
# for path in image_paths:
#   i = 0
#   for image_path in os.listdir(path):
#         input_path = os.path.join(path, image_path)
#         image_to_rotate = Image.open(input_path)
#         fullpath = os.path.join(path, 'rotated_'+ str(i) + '_' +image_path)
#         rotated = image_to_rotate.rotate(90).save(fullpath)
#         i = i + 1

**Flipping the images to increase the size of the dataset**

In [0]:
# image_paths = ["/content/drive/My Drive/data/with_mask", "/content/drive/My Drive/data/without_mask"]
# for path in image_paths:
#   i = 0
#   for image_path in os.listdir(path):
#         input_path = os.path.join(path, image_path)
#         image_to_flip = Image.open(input_path)
#         fullpath = os.path.join(path, 'flipped_'+ str(i) + '_' +image_path)
#         out = image_to_flip.transpose(PIL.Image.FLIP_LEFT_RIGHT).save(fullpath)
#         i = i + 1

**Making thee directory structure required by the image data generator keras**

In [0]:
# subdirs = ['train/', 'test/']
# for dir in subdirs:
#     labeldirs = ['with_mask/', 'without_mask/']
#     for labeldir in labeldirs:
#         newdir = '/content/drive/My Drive/train_data/' + dir + labeldir
#         os.makedirs(newdir, exist_ok=True)

Splitting the data (80-20 split) and putting them in the required directories under their class labels 

In [0]:
# seed(1)
# test_ratio = 0.20
# def category_split():
#   image_paths = ["with_mask", "without_mask"]
#   for directory in image_paths:
#     path_dir = '/content/drive/My Drive/data/' + directory
#     for file in os.listdir(path_dir):
#         type_dir = "train/"
#         main_dir = ''
#         if (random() < 0.20):
#             type_dir = "test/"
#         if (directory == "with_mask"):
#             main_dir = '/content/drive/My Drive/train_data/' + type_dir + 'with_mask/'
#         if (directory == "without_mask"):
#             main_dir = '/content/drive/My Drive/train_data/' + type_dir + 'without_mask/'
#         os.replace(path_dir + '/' + file, main_dir + file)

# category_split()

**Creating a simple convolutional network using 2 convolution layers with 3x3 filters**

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

**Passing the data from the directory using the generator object to reduce RAM usage**

In [0]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow_from_directory('/content/drive/My Drive/train_data/train', target_size= (150, 150), batch_size = 64)
test_it = datagen.flow_from_directory('/content/drive/My Drive/train_data/test', target_size = (150, 150), batch_size = 64)

Found 4408 images belonging to 2 classes.
Found 1102 images belonging to 2 classes.


**Fitting the model**

In [0]:
# Fitting the model
print("Fitting the model....")
history = model.fit_generator(train_it, steps_per_epoch = len(train_it), validation_data= test_it, validation_steps= len(test_it), epochs = 30, verbose = 2)

Fitting the model....
Epoch 1/30


**Saving the weights**

In [0]:
print("Saving the model")
model.save_weights('/content/drive/My Drive/saved_model/model_wieghts_latest.h5')
model.save('/content/drive/My Drive/saved_model/model_keras_latest.h5')

**TO load the local webcam in google colab**

**Predicting the images in the test set using the openCV library**

In [0]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
predict_path = "/content/drive/My Drive/prediction_images/all_classes/"
model = tf.keras.models.load_model("/content/drive/My Drive/saved_model/model_keras_150.h5")
size = 1
classifier = cv2.CascadeClassifier('/content/drive/My Drive/haarcascade_frontalface_default.xml') 

for file in os.listdir(predict_path):
    im = cv2.imread(predict_path + file)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray, 1.1, 1)

    for (x, y, w, h) in faces:
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2_imshow(im)
    cv2.imwrite(file, im)
    key = cv2.waitKey(10)
    if key == 27:
        break

cv2.destroyAllWindows()